In [1]:
import sys
import os
import json
import pickle
from PIL import Image
from tqdm import tqdm

# Add src to path
sys.path.append(os.path.abspath('..'))

from src.config import TARGET_BRANDS
from src.detection import ProductDetector
from src.classification import load_clip_model, classify_visual_nn
from src.graph_builder import get_spatial_relationships, build_scene_graph, visualize_polished_static
from src.utils import save_json

# set up directories
TEST_IMAGES_DIR = "../data/main-pipeline-testing-dataset"
OUTPUT_DIR = "../results"
WEIGHTS_DIR = "../weights"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "graphs"), exist_ok=True)

In [2]:
# load detector
# points to the model
detector_weights = os.path.join(WEIGHTS_DIR, "yolov11l best model.pt")
if not os.path.exists(detector_weights):
    print(f"Warning: {detector_weights} not found. Using intial finetuned YOLOv11n for testing.")
    detector_weights = os.path.join(WEIGHTS_DIR, "yolov11n initial finetuning.pt")

detector = ProductDetector(detector_weights)

# load clip model
# needed to encode the new crops from the shelf images
clip_model, clip_processor = load_clip_model()

# load visual anchors
# loads the memory bank created in notebook 02
anchors_path = os.path.join(WEIGHTS_DIR, "visual_anchors.pkl")

if os.path.exists(anchors_path):
    with open(anchors_path, 'rb') as f:
        data = pickle.load(f)
        ref_features = data['features'].to(clip_model.device)
        ref_labels = data['labels']
    print(f"Loaded {len(ref_labels)} visual anchors from file.")
else:
    raise FileNotFoundError(f"Anchors not found at {anchors_path}. Run Notebook 02 first.")

Loading Detection Model: ../weights/yolov11l best model.pt...
Detected YOLO architecture.
Loading CLIP model: openai/clip-vit-base-patch32...
Loaded 3712 visual anchors from file.


In [3]:
# processing loop
test_files = [f for f in os.listdir(TEST_IMAGES_DIR) if f.endswith(('.jpg', '.png', '.jpeg'))]
all_detections = []

if not test_files:
    print("No test images found. Upload images to data/test_images.")
else:
    print(f"Starting Pipeline on {len(test_files)} images...")
    
    # Create the progress bar
    pbar = tqdm(test_files, unit="img")
    
    for img_file in pbar:
        # Update the bar text to show which file is being processed
        pbar.set_description(f"Processing {img_file[:20]}...") 
        
        img_path = os.path.join(TEST_IMAGES_DIR, img_file)
        
        # detect products
        det_result = detector.detect(img_path)
        image = det_result['original_image']
        boxes = det_result['boxes']
        crops = det_result['crops']
        
        # classify products
        # compares shelf crops against the loaded anchors
        labels = []
        confidences = []
        
        for crop in crops:
            label, conf = classify_visual_nn(
                crop, clip_model, clip_processor, ref_features, ref_labels
            )
            labels.append(label)
            confidences.append(conf)
            
        # build scene graph
        w, h = image.size
        rels = get_spatial_relationships(boxes, labels, w, h)
        G = build_scene_graph(boxes, labels, confidences, rels, TARGET_BRANDS)

        # uncomment only if all 200+ graphs need to be displayed (NOT RECOMMENDED)
        # save visualization
        # graph_path = os.path.join(OUTPUT_DIR, "graphs", f"graph_{img_file}.html")
        # visualize_polished_static(G, graph_path)
        
        # collect results
        all_detections.append({
            "image": img_file,
            "boxes": boxes,
            "labels": labels,
            "confidences": confidences,
            "relationships": rels
        })

    # save final data for evaluation
    save_json(all_detections, os.path.join(OUTPUT_DIR, "all_detections.json"))
    print("\nPipeline complete. Results saved to all_detections.json")

Starting Pipeline on 211 images...


Processing fdc15e0a-c687-45d7-a...: 100%|██████████| 211/211 [08:04<00:00,  2.30s/img]


Saved JSON to: ../results/all_detections.json

Pipeline complete. Results saved to all_detections.json
